# StyleGan2 Requires TF 1.x

In [ ]:
%tensorflow_version 1.x
!pip install numpy==1.19

import numpy as np
np.version.version

TensorFlow 1.x selected.


'1.19.0'

# Connect to Drive

In [ ]:
from google.colab import drive
from pathlib import Path

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

Mounted at /content/drive


# Setup Project Structure

In [ ]:
repo = 'https://github.com/NVlabs/stylegan2-ada' # GAN we are going to retrain
root = drive_path / 'MyDrive' / 'cs1430-final' # root folder of our project
reset = False # change this if you imported a new images.zip file

if not root.is_dir(): # create root folder
  %mkdir $root

%cd $root

data_dir = root / 'data'          # dir paths
res_dir = root / 'results'
gan_dir = root / 'stylegan2-ada'

# create the dirs in project setup

if reset:
  %rm -rf $data_dir && rm -rf $res_dir

if not data_dir.is_dir(): 
  %mkdir $data_dir

if not res_dir.is_dir(): 
  %mkdir $res_dir

if not gan_dir.is_dir(): 
  !git clone $repo

/content/drive/MyDrive/cs1430-final


## **Important:** drag the images.zip file into the root folder in your drive before moving on

# Load Dataset

In [ ]:
import zipfile

if not (data_dir / 'images').is_dir(): # if we have not unzipped the images yet
  print("Extracting images...")

  with zipfile.ZipFile(str(root / ('images.zip')), 'r') as zip_ref: # unzip images into ./data/images
    zip_ref.extractall(str(data_dir))
    
  print('Extraction completed')
else:
  print("Images already exist!")

Extracting images...
Extraction completed


Convert to TF Records

In [ ]:
"""
This is the data format required by the GAN.
This cell will convert our images into the tf records.
"""

image_dir = data_dir / 'images'
tfr_dir = data_dir / 'tfr'
regenerate = True # change this if you imported a new images.zip file
if tfr_dir.is_dir() and regenerate:
    %rm -rf $tfr_dir
    %mkdir $tfr_dir
elif not tfr_dir.is_dir(): 
  %mkdir $tfr_dir    

!python stylegan2-ada/dataset_tool.py create_from_images $tfr_dir $image_dir

Loading images from "/content/drive/MyDrive/cs1430-final/data/images"
Creating dataset "/content/drive/MyDrive/cs1430-final/data/tfr"
stylegan2-ada/dataset_tool.py:96: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 102 images.


# Train

In [ ]:
snapshot_count = 2 # how often should the model generate samples and a .pkl file
mirrored = True # should the images be mirrored left to right?
mirroredY = False # should the images be mirrored top to bottom?
metric_list = None # metrics?
augs = 'bgc' # augments

resume_from = 'ffhq1024'

!python stylegan2-ada/train.py \
    --outdir=$res_dir \
    --data=$tfr_dir \
    --resume=$resume_from \
    --snap=$snapshot_count \
    --augpipe=$augs \
    --mirror=$mirrored \
    --metrics=$metric_list


python3.7 stylegan2-ada/train.py --outdir=res --data=data/tfr --resume='ffhq1024' --snap=2 --augpipe='bgc' --mirror=true --metrics=None

tcmalloc: large alloc 4294967296 bytes == 0x6b42000 @  0x7fe1b269d001 0x7fe1af8c41af 0x7fe1af91ac23 0x7fe1af91ba87 0x7fe1af9bd823 0x5936cc 0x548c51 0x5127f1 0x549e0e 0x4bca8a 0x532b86 0x594a96 0x548cc1 0x5127f1 0x549576 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x549e0e 0x4bca8a 0x5134a6 0x593dd7 0x5118f8 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2
tcmalloc: large alloc 4294967296 bytes == 0x7fdff539a000 @  0x7fe1b269b1e7 0x7fe1af8c40ce 0x7fe1af91acf5 0x7fe1af91af4f 0x7fe1af9bd673 0x5936cc 0x548c51 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x593dd7 0x5118f8 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x549e0e
tcmalloc: large alloc 4294967296 bytes == 0x7fdef4398000 @  0x7fe1b269b1e7 0x7fe1af8c40ce 0x7fe1af91acf5 0x7fe1af91af4f 0x7fe172ef0235 0x7fe172873792 0x7fe172873d42 0x7fe17282caee 0x59371f 0x548c51 0x51566f 0x593dd7 0x511e2c 0x549e0e 0x4bcb19 0x5134a6 0x549576 0x593f

# Generate

In [ ]:
%pip install opensimplex

  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.0
    Uninstalling numpy-1.19.0:
      Successfully uninstalled numpy-1.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lucid 0.3.10 requires umap-learn, which is not installed.
tensorflow 1.15.2 requires gast==0.2.2, but you have gast 0.5.3 which is incompatible.
lucid 0.3.10 requires numpy<=1.19, but you have numpy 1.21.6 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
!python stylegan2-ada/generate.py -h

usage: generate.py [-h] --network NETWORK_PKL
                   (--seeds SEEDS | --dlatents DLATENTS_NPZ)
                   [--trunc TRUNCATION_PSI] [--class CLASS_IDX] --outdir DIR

Generate images using pretrained network pickle.

optional arguments:
  -h, --help            show this help message and exit
  --network NETWORK_PKL
                        Network pickle filename
  --seeds SEEDS         List of random seeds
  --dlatents DLATENTS_NPZ
                        Generate images for saved dlatents
  --trunc TRUNCATION_PSI
                        Truncation psi (default: 0.5)
  --class CLASS_IDX     Class label (default: unconditional)
  --outdir DIR          Where to save the output images

examples:

  # Generate curated MetFaces images without truncation (Fig.10 left)
  python generate.py --outdir=out --trunc=1 --seeds=85,265,297,849 \
      --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/metfaces.pkl

  # Generate uncurated MetFaces images with truncati

In [ ]:
from numpy import random
seed_init = random.randint(10000)
nbr_images = 6 - 1
seeds = "{}-{}".format(seed_init, seed_init + nbr_images) #seed_init + "-" + seed_init + nbr_images


generation_from = res_dir / 'model' / 'res.pkl'

!python stylegan2-ada/generate.py \
    --outdir=$res_dir \
    --trunc=0.7 \
    --seeds=0-10 \
    --network=$generation_from

Loading networks from "/content/drive/MyDrive/cs1430-final/results/model/res.pkl"...
Traceback (most recent call last):
  File "stylegan2-ada/generate.py", line 121, in <module>
    main()
  File "stylegan2-ada/generate.py", line 116, in main
    generate_images(**vars(args))
  File "stylegan2-ada/generate.py", line 27, in generate_images
    with dnnlib.util.open_url(network_pkl) as fp:
  File "/content/drive/MyDrive/cs1430-final/stylegan2-ada/dnnlib/util.py", line 386, in open_url
    return url if return_filename else open(url, "rb")
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/cs1430-final/results/model/res.pkl'
